In [1]:
from pydantic import BaseModel, Field

class CustomerName(BaseModel):
    """Greet the customer with the customer's name in the greeting message. If not explicitly told that the name is wrong or wrong name dialed, assume the name is confirmed.
    Example: 
        Customer Service Representative: "喂，您好，请问是xxx吗?"
        Customer: "/呃/你说/是的/哪里/咋了/哦/是/嗯...."  -> create CustomerName(name="xxx") # all the ambiguous response should be treated as confirmation
        Customer: "/不是/打错了/我不是" -> do not create
    """
    name: str = Field(..., description = 'The name of the customer')
    
    
    def transfer(self) -> str:
        print("Transferring to FinancialSupportStatus")
        return 'financial_support_inquiry'
    
    
class FinancialSupportStatus(BaseModel):
    """Whether the customer need financial support. True is the customer do not refuse/reject financial support.
    As long the the customer asking product details or show insterest, create FinancialSupportInquiry(require_support=True).
    Explicit rejection or refusal should create FinancialSupportInquiry(require_support=False).
    Other case, you should continue to ask the customer for clarification.
    """
    require_support: bool = Field(..., description = 'Whether the customer need financial support')
    
    def transfer(self) -> str:
        print("Transferring to VehicleNotUnderRepayment")
        return 'vehicle_payment_status'
    
    
class VehicleNotUnderRepayment(BaseModel):
    """Whether the vehicle is not under repayment.
    If the vehicle is fully paid off, create VehicleNotUnderRepayment(is_not_under_repayment=True).
    If the vehicle is still under repayment, create VehicleNotUnderRepayment(is_not_under_repayment=False).
    """
    is_not_under_repayment: bool = Field(..., description = 'Whether the vehicle is not under repayment')
    
    
    def transfer(self) -> str:
        print("Transferring to VehicleLiscenceUnderControl")
        return 'vehicle_liscence_under_control'
    
class VehicleLiscenceUnderControl(BaseModel):
    """Whether the vehicle liscence(Chinese机动车行驶证， 绿本， 大本) is under the customer's control.
    If the vehicle liscense is under the customer's control, create VehicleLiscenceUnderControl(is_under_control=False).
    If the vehicle liscense is under the customer's spouse or family member's control, create VehicleLiscenceUnderControl(is_under_control=True).
    If the vehicle liscense ir under his/her company's control, create VehicleLiscenceUnderControl(is_under_control=False).
    """
    
    is_under_control: bool = Field(..., description = 'Whether the vehicle liscence is under the customer\'s control')
    
    def transfer(self) -> str:
        print("Transferring to AgreeToAddWechatAccount")
        return "agree_wechat_add"


class AgreeToAddWechatAccount(BaseModel):
    """Whether the customer agree to add wechat account for further contact.
    If the customer agree to add wechat account, create AgreeToAddWechatAccount(agree=True).
    If the customer refuse to add wechat account, create AgreeToAddWechatAccount(agree=False).
    """
    agree: bool = Field(..., description = 'Whether the customer agree to add wechat account')
    
    
    def transfer(self) -> str:
        print("Transferring to WeChatId")
        return "ask_wechat_id"
    
    
class WeChatId(BaseModel):
    """The WeChat ID provided by the customer for further contact.
    Example:
        Customer Service Representative: "请问您的微信号是多少？"
        Customer: "我的微信号是abc123" -> create WeChatId(wechat_id="abc123")
    """
    wechat_id: str = Field(..., description = 'The WeChat ID provided by the customer')
    
    
    def transfer(self) -> str:
        print("Transferring to HangUp")
        return 'hangup'

In [6]:
from voice_agent_flow.agents.agent_node import AgentNode
from voice_agent_flow.llms import create_pydantic_azure_openai

from voice_agent_flow.agents.agent_node import AgentNode, HangUpNode, DoHangUp

from voice_agent_flow.agents.multi_agent_runner import MultiAgentRunner

model = create_pydantic_azure_openai('gpt-4o-mini')

instruction = """
You are a customer service representative in a auto finance company. You task is to talk with customer via telephone to collection information.
You will be given one task defined by a schema, you should talk with the user until you can fill the schema completely.
You shoule speak in Chinese.
On each text reply, you should output as less text as possible to collect information.

Overall Conversation Policy: For yes or not question, if the customer's response is not explicitly reject or refuse, you should assume the customer agree or accept it.
Then create the schema with the corresponding fields filled.

Note you are part of a multi-agent system, do not add additional explaination, greeting or closing statement, just focus on collecting information to fill the schema.
Do not Generate Structured output until you have collected information defined by the schema. Before that ,you shoule chat with the customer.

"""

agents = {
    
    # Complex business rules, you need more prompt, but just in this step.
    "customer_name_inquiry": AgentNode(
        name="customer_name_inquiry",
        model=model,
        instruction=instruction,
        task_cls= CustomerName,
        step_instruction=(
            "Confirm the customer's name with a greeting message(customer name included in the message)."
            "Amubiguous response from customer should be treated as confirmation and create the schema. Current Customer Name: 李老三"
            "Any response for the message will be treated as confirmation unless the customer explicitly says he/she is not the person or dialed wrong number."
            "Event a simple ‘嗯’, ‘呃’, '哪里'，‘你说’ indicates a confirmation. You shoule create the schema immediately"
        ),
            
        examples=["您好，请问是xxx(plug customer name here)吗？"],
        ),
    
    # Complex business rules, you need more prompt, but just in this step.
    "financial_support_inquiry": AgentNode(
        name="financial_support_inquiry",
        model=model,
        instruction=instruction,
        task_cls= FinancialSupportStatus,
        step_instruction=(
            "Ask the customer whether he/she need financial support."
            "Customer raising a new question in step indicates she/he is interested in the financial support."
            "You should answer the customer query first, after the deviation is handled, create the schema immediately."
        ),
        examples=["您好，李先生，请问您对我们的金融支持服务感兴趣吗？"],
        ),
    
    # simple business rules, you can be direct and concise.
    "vehicle_payment_status": AgentNode(
        name="vehicle_payment_status",
        model=model,
        instruction=instruction,
        task_cls= VehicleNotUnderRepayment,
        step_instruction="Ask the customer whether the vehicle is already fully paid off.",
        examples=["请问您的车辆目前是已经还清贷款了吗？"],
        ),
    
    # simple business rules, you can be direct and concise.
    "vehicle_liscence_under_control": AgentNode(
        name="vehicle_liscence_under_control",
        model=model,
        instruction=instruction,
        task_cls= VehicleLiscenceUnderControl,
        step_instruction="Ask the customer whether the vehicle liscence is under the customer's control.",
        examples=["请问您的车辆行驶证现在是在您本人手上吗？"],
        ),
    
    # simple business rules, you can be direct and concise.
    "agree_wechat_add": AgentNode(
        name="agree_wechat_add",
        model=model,
        instruction=instruction,
        step_instruction="Ask the customer whether he/she agree to add wechat account for further contact.",
        task_cls= AgreeToAddWechatAccount,
        examples=["为了方便后续联系，您是否同意添加我们的微信账号？"],
        ),
    
    # simple business rules, you can be direct and concise.
    "ask_wechat_id": AgentNode(
        name="ask_wechat_id",
        model=model,
        instruction=instruction,
        step_instruction="Ask the customer for his/her wechat id.",
        task_cls= WeChatId,
        examples=["请问您的微信号是多少？"],
        ),
    
    "hangup": HangUpNode(model = model)
}



In [11]:
from voice_agent_flow.agents import AgentSession

In [12]:
runner = MultiAgentRunner(
    agents=agents, 
    entry_agent_name="customer_name_inquiry",
    ending_message="好的，我们稍后会加您的微信，请你注意在服务通知后查看我们的企业微信请求，再见！"
)  
chat = AgentSession(runner)

In [13]:
_ = await chat.chat("喂")

🤖[customer_name_inquiry]...Working.
您好，请问是李老三吗？

In [14]:
_ = await chat.chat("嗯嗯")

🤖[customer_name_inquiry]...Working.
Transferring to FinancialSupportStatus
AgentHandoff(status=None, message={'source_agent_name': 'customer_name_inquiry', 'target_agent_name': 'financial_support_inquiry'})
请问您对我们的金融支持服务感兴趣吗？

In [15]:
_ = await chat.chat("有兴趣")

🤖[financial_support_inquiry]...Working.
请问您想了解哪些具体的产品信息呢？

In [16]:
_ = await chat.chat("什么？")

🤖[financial_support_inquiry]...Working.
您对金融支持服务有哪些疑问？比如贷款利率、还款方式等。

In [17]:
_ = await chat.chat("没啥疑问？")

🤖[financial_support_inquiry]...Working.
Transferring to VehicleNotUnderRepayment
AgentHandoff(status=None, message={'source_agent_name': 'financial_support_inquiry', 'target_agent_name': 'vehicle_payment_status'})
好的。请问您的车辆目前是已经还清贷款了吗？

In [18]:
_ = await chat.chat("还清了？")

🤖[vehicle_payment_status]...Working.
Transferring to VehicleLiscenceUnderControl
AgentHandoff(status=None, message={'source_agent_name': 'vehicle_payment_status', 'target_agent_name': 'vehicle_liscence_under_control'})
是的，请问您的车辆行驶证现在是在您本人手上吗？

In [19]:
_ = await chat.chat("在我手里")

🤖[vehicle_liscence_under_control]...Working.
Transferring to AgreeToAddWechatAccount
AgentHandoff(status=None, message={'source_agent_name': 'vehicle_liscence_under_control', 'target_agent_name': 'agree_wechat_add'})
为了方便后续联系，您是否同意添加我们的微信账号？

In [20]:
_ = await chat.chat("方便的")

🤖[agree_wechat_add]...Working.
Transferring to WeChatId
AgentHandoff(status=None, message={'source_agent_name': 'agree_wechat_add', 'target_agent_name': 'ask_wechat_id'})
请问您的微信号是多少？

In [21]:
_ = await chat.chat("15001395923")

🤖[ask_wechat_id]...Working.
Transferring to HangUp
AgentHandoff(status=None, message={'source_agent_name': 'ask_wechat_id', 'target_agent_name': 'hangup'})
非常感谢您提供信息。现在我将与您确认下，是否还有其他问题需要帮助的？如果没有，我们就要结束这个电话了。再见！

In [22]:
_ = await chat.chat("再见")

🤖[hangup]...Working.
HangupSignal(status=None, message=DoHangUp())
Conversation Ended with Hangup Signal.
